**This is the simplified Opensantions data in csv format**

In [1]:
#import packages
import requests
import pandas as pd
import numpy as np
import os

# OFAC SDN

In [2]:
#I can access the data via pandas very easily, output already in tabular form
data = pd.read_csv('https://data.opensanctions.org/datasets/20240121/us_ofac_sdn/targets.simple.csv')
data

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,last_change
0,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,Nairobi,NaN,SDN List - Executive Order 13818 (Global Magni...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
1,NK-22oMG6jqPQknWaMjzTn4hK,Organization,Limited Liability Company Garantiya,Garantiya OOO,NaN,ru,"bulvar Tverskoi, d. 15 str. 2, Moscow",5067746901426;7703610362,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
2,NK-23p2d4vMT5sJtQ845GyzJt,Organization,Scientific and Production Association of Measu...,AO NPO IT;Aktsionernoe Obschestvo Nauchno Proi...,NaN,ru,"2k4 Pionerskaya Str., 141074 Korolyov",1095018006555;5018139517,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
3,NK-23rgYEXa9AHtupZKgS8Tbc,Person,Bahram Ali SHAYESTEH,Bahrami Ali JADALI;Bahrami Ali SHAYESTEH,1958-06-13;1963-05-06;1963-08-06,de,"80331 Muenchen, Bayern",NaN,SDN List - Unknown,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
4,NK-24UMVkuxF3svsAtg55i3Zi,Organization,ALBAHR ALAAHMAR ENERGY FZE,البحر الأحمر للطاقة م م ح,NaN,ae,"P1-ELOB Office No. E-44G-29, Hamriyah Free Zon...",11670819;19438,SDN List - Executive Order 13846 (Iran),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13799,ofac-47225,Vessel,Peria,NaN,NaN,lr,NaN,5LIZ6;636022479;IMO 9322827,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01
13800,ofac-47226,Vessel,Sara II,NaN,NaN,lr,NaN,5LJI4;636022546;IMO 9301615,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01
13801,ofac-47227,Vessel,Sensus,NaN,NaN,lr,NaN,5LHJ7;636022146;IMO 9296585,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01
13802,ofac-47228,Vessel,Uze,NaN,NaN,lr,NaN,5LHB3;636022072;IMO 9323338,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01


In [3]:
#create urls for all relevant dates
date_list = pd.date_range(start='20220101',end='20220114',freq='D').strftime('%Y%m%d')
date_list

Index(['20220101', '20220102', '20220103', '20220104', '20220105', '20220106',
       '20220107', '20220108', '20220109', '20220110', '20220111', '20220112',
       '20220113', '20220114'],
      dtype='object')

In [29]:
#get all the urls for all the dates we need the data for
#make each day an individuallist so we can compare and match them
websites = []

for i in date_list:
    test = 'https://data.opensanctions.org/datasets/'+(i)+'/us_ofac_sdn/targets.simple.csv'
    websites.append(test)
print(websites)

['https://data.opensanctions.org/datasets/20220101/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220102/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220103/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220104/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220105/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220106/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220107/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220108/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220109/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220110/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220111/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20220112/us_ofac_sdn/targets.simple.csv', 'ht

In [ ]:
import re
import pandas as pd

entities_list = []
date_pattern = r'/datasets/(\d{8})/'

for site in websites:
    data = pd.read_csv(site)
    
    # Extract date from URL using regex
    match = re.search(date_pattern, site)
    if match:
        date = match.group(1)
        data['date_stamp'] = date
        data = data.set_index('date_stamp')
        
        entities_list.append(data)

res = pd.concat(entities_list)

In [45]:
import re

In [54]:
#now loop over the batches to get all the data and append the dataframes to one another
#this gives us a list of dataframes
#the loop must contain a date identifyer as a new column so each dataframe is marked with the retrieval date that is then needed to match the different days against each other
entities_list = []
date_pattern = r'/datasets/(\d{8})/'
for site in websites:
    data = pd.read_csv(site)
    match = re.search(date_pattern, site) #extract the date from the url
    if match:
        date = match.group(1)
        data['date_stamp'] = date #set an individual date stamp for each dataframe matching the url date
        data =  data.set_index('date_stamp') #set as an index
    entities_list.append(data)
res = pd.concat(entities_list)  # concatenate list of dataframes

In [55]:
res

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,last_seen,first_seen
date_stamp,,,,,,,,,,,,,,
20220101,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,"Nairobi, 248-00100;South Sudan",NaN,SDN List - Program - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-01 03:04:33
20220101,NK-23rgYEXa9AHtupZKgS8Tbc,Person,Bahram Ali SHAYESTEH,Bahrami Ali JADALI;Bahrami Ali SHAYESTEH,1958-06-13;1963-05-06;1963-08-06,de,"80331 Muenchen, Bayern",NaN,SDN List - Program - Block - Unknown - 2011-04-28,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-01 03:04:33
20220101,NK-24KmksG96rQedGYXzm4xHU,Organization,CONSTRUCCIONES E INVERSIONES LTDA.,NaN,NaN,co,"Calle 15 No. 10-52, La Union, Valle",800154939-3,SDN List - Program - Block - Unknown - 2006-11-28,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-01 03:04:33
20220101,NK-24cEBmXGjPPVJRwMPGx3Yj,Company,"LOLA LOLITA 1110, S. DE R.L. DE C.V.","LOLALOLITA 1110, S. DE R.L. DE C.V.",NaN,mx,"Av. Vallarta 1110, Col. Americana, 44160 Guada...",60645 (Jalisco),SDN List - Program - Block - Foreign Narcotics...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-01 03:04:33
20220101,NK-24kqBFxEsFzjqBYhaNHkC9,Organization,TAWU Mechanical Engineering and Trading Company,Metaalkunde BV;TAWU;TAWU BVBA;TAWU BVBA Mechan...,NaN,be,"Bleidenhoek 34, 2230 Herselt",686896689;BE0686.896.689;BE0686896689,SDN List - Program - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-01 03:04:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220114,ofac-34279,Company,Enterprise Comercio de Moveis e Intermediacao ...,Caso e Casa;Enterprise Comércio de Moveis e In...,NaN,br,"Rua Ernesto Nazareth 18, Jardim Paraventi, Gua...",35.116.112/0001-97,Block - SDN List - Program - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-12-23 03:03:37,2022-01-14 03:04:32
20220114,ofac-34280,Company,Home Elegance Comercio de Moveis Eireli,Daiana Portella Coelho Comercio de Moveis e Co...,NaN,br,"Rua Dorezopolis, 669, Casa 03, Jardim Santa Cl...",31.746.200/0001-11,Block - SDN List - Program - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-12-23 03:03:37,2022-01-14 03:04:32
20220114,ofac-34456,Person,Yong Ho O,NaN,1961-12-25,kp;ru,"Korea, North;Moscow",108410041,Block - SDN List - Program - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2022-01-13 03:05:01,2022-01-14 03:04:32


In [ ]:
#now match the different dates against each other and return an indicator per row for a new listing or delisting